In [ ]:
import pandas as pd
import transform_files as transform
from fastparquet import write
import requests
import json
from pandas.io.json import json_normalize

df_happiness = pd.read_csv('happiness.csv', index_col=0, header=0)
df_country = pd.read_csv('country.csv', index_col=0, header=0)
df_happiness['Happiness Status'] = transform.status_calc(df_happiness)

#api fetch
api_url = "http://api.worldbank.org/v2/country?format=json"
# store the response of URL
response = requests.get(api_url)
print(response.status_code)
data = response.json()
# display(data)
a = json_normalize(data[1],sep="-")
# display(list(a))

# create dataframe with the json responsejsonon_data "nested_array_to_expand"
df_json = json_normalize(data[1],sep="-",meta=["name","capitalCity","longitude","latitude"])
df_json = df_json[['capitalCity', 'longitude',"latitude","name"]]
# display(df_json)

data = [df_happiness["Year"],df_happiness["Country"], df_country["image_url"],df_country["region-code"],df_country["region"].str.upper()
       ,df_happiness['Year'].rank(),df_happiness["Happiness Score"],df_happiness["GDP per capita"],df_happiness["Family"],df_happiness["Social support"],
        df_happiness["Healthy life expectancy"],df_happiness["Freedom"],df_happiness["Generosity"],df_happiness["Perceptions of corruption"],
        df_happiness["Happiness Status"]]
headers = ["Year","Country", "Country_url","Region Code","Region","Overall Rank","Happiness Score","GDP per capita","Family","Social support",
          "Healthy life expectancy","Freedom to make life choices","Generosity","Perceptions of corruption","Happiness Status"]
df3 = pd.concat(data, axis=1, keys=headers)
df3['Region Code'] = df3['Region Code'].fillna(0)
df3['Overall Rank'] = df3['Overall Rank'].fillna(0)
df3 = df3.astype({"Region Code":"int","Overall Rank":"int"})
# join the new dfjson and df3 on name and country and create a new df_final
df_final= df_json.merge(df3, how="left", left_on=['name'], right_on=['Country'])
df_final.drop(['name'], axis = 1,inplace=True)
# display(df_final)
#save to new csv and parquet file
df_final.to_csv('report_one.csv',index=False)
write('report_one.parq', df_final)
# display(df3.groupby('Year').count())
# display(df3)